### 🧠 Что такое экзогенные переменные во временных рядах?

**Экзогенные переменные (exogenous variables)** — это внешние по отношению к анализируемому временному ряду факторы, которые могут влиять на его поведение, но сами **не зависят** от этого ряда.

Примеры:
- Ты прогнозируешь спрос на электричество. Температура воздуха — экзогенная переменная.
- Прогнозируешь количество выпусков сертификатов. Праздники, количество рабочих дней, акции или внутренние регламенты могут быть экзогенными.

Во временных рядах это обозначается как **X** — внешние переменные, подаваемые в модель наряду с основным рядом **Y**.

---

### 📈 Как называется модель, где есть и лаги, и экзогенные переменные?

Это будет:

#### **ARIMAX** (или SARIMAX, если с сезонностью) — AutoRegressive Integrated Moving Average with eXogenous variables.

- **AR** — авторегрессия (использует лаги Y).
- **I** — интегрированная часть (разности, если ряд нестационарен).
- **MA** — скользящее среднее.
- **X** — экзогенные переменные (внешние факторы).

Аналогично в других моделях:
- В Prophet: `add_regressor()`
- В XGBoost/CatBoost: просто подаются лаги и экзогенные факторы как обычные фичи.
- В нейросетях (например, LSTM): можно подать лаги и экзогенные переменные как входной вектор.

---

### 🛠️ Немного терминов:

| Термин | Описание |
|-------|----------|
| **Эндогенные переменные** | Зависимые от самого ряда (например, лаги самого Y). |
| **Экзогенные переменные** | Внешние, независимые от Y. |
| **ARIMA** | Только лаги (эндогенные). |
| **ARIMAX/SARIMAX** | Лаги + экзогенные переменные. |

---

Если хочешь — можем прямо сейчас взять пример временного ряда и попробовать добавить экзогенные переменные в SARIMAX. Или могу показать пример на `statsmodels` / `pmdarima`. Готов?

### **1. Понимание данных**
Перед началом анализа важно убедиться, что данные готовы для работы. Ответь на следующие вопросы:
- Как часто собираются данные (ежедневно, еженедельно, ежечасно)?
- Есть ли пропущенные значения или аномалии?
- Какие метрики ты хочешь анализировать? Например:
  - Количество выданных сертификатов за день/неделю.
  - Количество отозванных сертификатов.
  - Соотношение выданных и отозванных сертификатов.

Если данные есть в формате CSV или Excel, их можно загрузить в Python с помощью библиотеки `pandas`.

Пример кода для загрузки данных:
```python
import pandas as pd

# Загрузка данных из CSV
data = pd.read_csv('certificates_data.csv')

# Если дата не является индексом, преобразуем её
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# Просмотр первых строк
print(data.head())
```

---

### **2. Исследование временного ряда**
На этом этапе мы проверяем основные характеристики данных:
- Тренд (рост или падение числа событий во времени).
- Сезонность (циклические паттерны, например, повышение активности в определённые дни недели или месяцы).
- Аномалии (необычные пики или провалы).

#### Пример визуализации:
```python
import matplotlib.pyplot as plt

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(data['issued_certificates'], label='Выданные сертификаты')
plt.plot(data['revoked_certificates'], label='Отозванные сертификаты')
plt.title('Анализ сертификатов во времени')
plt.xlabel('Дата')
plt.ylabel('Количество')
plt.legend()
plt.show()
```

---

### **3. Разложение временного ряда**
Чтобы лучше понять структуру данных, можно использовать метод разложения (`seasonal_decompose` из библиотеки `statsmodels`). Этот метод позволяет разделить временной ряд на три компоненты:
- **Тренд** — общее направление изменения данных.
- **Сезонность** — повторяющиеся паттерны.
- **Остаток** — случайные флуктуации.

Пример кода:
```python
from statsmodels.tsa.seasonal import seasonal_decompose

# Разложение временного ряда
decomposition = seasonal_decompose(data['issued_certificates'], model='additive', period=7)  # period=7 для недельной сезонности

# Визуализация компонент
plt.figure(figsize=(10, 8))
plt.subplot(411)
plt.plot(data['issued_certificates'], label='Исходные данные')
plt.legend()

plt.subplot(412)
plt.plot(decomposition.trend, label='Тренд')
plt.legend()

plt.subplot(413)
plt.plot(decomposition.seasonal, label='Сезонность')
plt.legend()

plt.subplot(414)
plt.plot(decomposition.resid, label='Остаток')
plt.legend()

plt.tight_layout()
plt.show()
```

Это поможет выявить:
- Есть ли долгосрочный тренд (например, рост числа выданных сертификатов).
- Какова частота сезонности (например, каждую неделю или месяц).

---

### **4. Моделирование и прогнозирование**
Для прогнозирования будущих значений можно использовать различные модели. Вот несколько подходящих вариантов:

#### a) **ARIMA/SARIMA**
Это классические модели для анализа временных рядов. SARIMA особенно полезна, если есть явная сезонность.

Пример кода:
```python
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Обучение модели SARIMA
model = SARIMAX(data['issued_certificates'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))  # 7 для недельной сезонности
results = model.fit()

# Прогноз на 30 дней вперёд
forecast = results.get_forecast(steps=30)
forecast_values = forecast.predicted_mean

# Визуализация прогноза
plt.figure(figsize=(10, 6))
plt.plot(data['issued_certificates'], label='Исторические данные')
plt.plot(forecast_values, label='Прогноз', color='red')
plt.title('Прогноз количества выданных сертификатов')
plt.legend()
plt.show()
```

#### b) **Prophet**
Prophet — это удобная библиотека от Facebook, которая хорошо работает с данными, имеющими сезонность и тренды.

Пример кода:
```python
from fbprophet import Prophet

# Подготовка данных для Prophet
df = data.reset_index()
df.columns = ['ds', 'y']

# Создание и обучение модели
model = Prophet(seasonality_mode='additive', weekly_seasonality=True)
model.fit(df)

# Прогноз на 30 дней вперёд
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Визуализация результатов
model.plot(forecast)
plt.show()
```

#### c) **Машинное обучение (LSTM)**
Если данные сложные и нелинейные, можно использовать рекуррентные нейронные сети (RNN/LSTM). Это более сложный подход, но он даёт хорошие результаты для многомерных временных рядов.

---

### **5. Оценка качества прогноза**
После построения модели важно оценить её точность. Для этого используются метрики, такие как:
- MAE (Mean Absolute Error): средняя абсолютная ошибка.
- RMSE (Root Mean Squared Error): корень из среднеквадратичной ошибки.
- MAPE (Mean Absolute Percentage Error): средняя абсолютная процентная ошибка.

Пример кода:
```python
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Расчёт ошибок
mae = mean_absolute_error(test_data, forecast_values)
rmse = mean_squared_error(test_data, forecast_values, squared=False)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
```

---

### **6. Детектирование аномалий**
В контексте кибербезопасности важно также находить аномалии (например, резкий скачок отзывов сертификатов). Для этого можно использовать методы, основанные на стандартных отклонениях или моделях машинного обучения.

Пример кода для простого детектора аномалий:
```python
# Вычисление границ нормальных значений
mean = data['issued_certificates'].mean()
std = data['issued_certificates'].std()
lower_bound = mean - 3 * std
upper_bound = mean + 3 * std

# Нахождение аномалий
anomalies = data[(data['issued_certificates'] < lower_bound) | (data['issued_certificates'] > upper_bound)]

print(anomalies)
```

---

Если у тебя есть конкретные данные, можем вместе поработать над их анализом. Что скажешь? 😊

## ✅ 1. Как выбрать модель, глядя на ACF и PACF?

### 📌 Обозначения:
- **ACF (Autocorrelation Function)** — автокорреляция: связь между текущим значением и прошлыми.
- **PACF (Partial ACF)** — частичная автокорреляция: чистая связь после учёта промежуточных лагов.

| Тип модели | Как выглядит ACF | Как выглядит PACF |
|------------|------------------|--------------------|
| **AR(p)** | экспоненциальное или колебательное убывание | обрывается после лага `p` |
| **MA(q)** | обрывается после лага `q` | экспоненциальное/затухающие колебания |
| **ARMA(p, q)** | и ACF, и PACF затухают | и ACF, и PACF затухают |
| **ARIMA(p,d,q)** | смотри ACF/PACF на **разностном** ряде | аналогично ARMA, но после дифференцирования |

🔍 Пример:
- Если **ACF резко обрывается** на лаге 2, а **PACF затухает** → вероятно, **MA(2)**.
- Если **PACF резко обрывается** на лаге 1, а **ACF затухает** → вероятно, **AR(1)**.

---

## ✅ 2. На что смотреть при выборе модели?

### Основные шаги:
1. **Проверка стационарности**:
   - Используем: ADF, KPSS, PP-тест.
   - Если ряд нестабилен — делаем дифференцирование (1-е, 2-е).

2. **ACF/PACF** — выбираем тип модели, как выше.

3. **Сравнение моделей по метрикам**:
   - AIC / BIC (чем меньше — тем лучше).
   - RMSE / MAE (на кросс-валидации).

4. **Анализ остатков**:
   - Должны быть как "белый шум".
   - Проверки: Ljung-Box (на автокорреляцию), ARCH (на гетероскедастичность).

---

## ✅ 3. Нужно ли анализировать остатки в ML-моделях (Prophet, LSTM, CatBoost и др.)?

### Краткий ответ: **да, но не всегда строго обязательно**.

📌 Вот как это работает:

| Модель | Анализ остатков нужен? | Зачем смотреть? |
|--------|-------------------------|-----------------|
| **Prophet** | Да | Проверить, что модель объяснила тренды/сезонность. Остатки — "шум"? |
| **LSTM** | Желательно | Найти недообучение/переобучение. Можно проверять autocorr(остатков). |
| **CatBoost / XGBoost** | Опционально | Важно при построении фичей. Остатки могут подсказать, чего не хватает. |

**Если остатки автокоррелированы** → это сигнал: в данных есть неучтённые паттерны, которые можно моделировать (например, добавить лаги или новые признаки).

**Если гетероскедастичность** → возможно, нужно использовать log-преобразования, нормализацию, сегментирование по кластерам и т.п.

---

## 🛠️ Практический совет:

После построения модели всегда хорошо сделать:
- График остатков.
- ACF/PACF остатков.
- Тест Бокса-Льюнга (на автокорреляцию).
- ARCH-тест (на гетероскедастичность).

---

Если хочешь — могу прямо сейчас показать на Python:
- как интерпретировать ACF/PACF по примеру,
- как протестировать остатки после модели ARIMA или CatBoost.

Готов глянуть на код?

### **1. Как понять по автокорреляционной функции (ACF) и частной автокорреляционной функции (PACF), какую модель выбрать?**

Для выбора модели ARIMA (или её компонент AR/MA) анализ ACF и PACF играет ключевую роль. Вот основные правила:

#### **АКФ (Autocorrelation Function)**:
- Показывает корреляцию временного ряда с его собственными запаздывающими значениями.
- Если корреляция медленно затухает (похоже на экспоненциальное затухание или синусоиду), это указывает на наличие тренда или сезонности.

#### **ЧАКФ (Partial Autocorrelation Function)**:
- Показывает корреляцию между временными моментами, исключая влияние промежуточных значений.
- Если ЧАКФ "резко обрывается" после определённого числа лагов, это помогает определить порядок AR-компонента.

---

#### **Правила для AR, MA, ARMA и ARIMA**:

| **Модель** | **ACF**                                                                                     | **PACF**                                                                                  |
|------------|---------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| **AR(p)**  | Медленно затухает (экспоненциально или в виде синусоиды).                                   | "Резко обрывается" после p лагов.                                                         |
| **MA(q)**  | "Резко обрывается" после q лагов.                                                           | Медленно затухает.                                                                       |
| **ARMA(p,q)** | Комбинация: медленное затухание + резкий обрыв после q лагов.                             | Комбинация: резкий обрыв после p лагов + медленное затухание.                            |
| **ARIMA(p,d,q)** | Аналогично ARMA, но данные должны быть стационаризированы через дифференцирование (d). | Аналогично ARMA, но данные должны быть стационаризированы через дифференцирование (d). |

---

#### **Пример анализа ACF/PACF**:
Предположим, у нас есть следующие графики:

1. **ACF**: Значения медленно затухают, что указывает на возможный тренд или необходимость дифференцирования.
2. **PACF**: Значения "обрываются" после 2 лагов.

Это может означать, что подходит модель **AR(2)** или **ARIMA(2,1,0)** (если нужно дифференцирование).

---

### **2. На что ещё смотреть при выборе модели?**

Если ACF/PACF не дают однозначного ответа, можно использовать дополнительные инструменты:

#### **a) Тесты на стационарность**
- Временной ряд должен быть стационарным для применения моделей ARIMA.
- Используй тесты Адфулера-Дикки (`ADF`) или КПСС (`KPSS`):
  - Если ряд нестационарен, применяй дифференцирование (увеличивай параметр `d`).

```python
from statsmodels.tsa.stattools import adfuller

result = adfuller(data['value'])
print('p-value:', result[1])
```

#### **b) Графический анализ**
- Посмотри на график данных: есть ли явные тренды или сезонность?
- Если есть сезонность, используй SARIMA (сезонная версия ARIMA).

#### **c) Информационные критерии**
- После обучения нескольких моделей (например, ARIMA(1,1,1), ARIMA(2,1,1)) сравни их по критериям AIC/BIC:
  - Модель с меньшим значением AIC/BIC считается лучше.

```python
print(model.aic)
print(model.bic)
```

#### **d) Прогнозирование и качество**
- Выбери ту модель, которая даёт лучший прогноз на тестовой выборке (меньшие ошибки MAE/RMSE).

---

### **3. Нужно ли анализировать остатки для моделей ML (Prophet, LSTM, XGBoost, CatBoost)?**

Да, анализ остатков важен даже для моделей машинного обучения. Вот почему:

#### **a) Автокорреляция в остатках**
- Если остатки имеют значительную автокорреляцию, это значит, что модель не учла часть зависимостей во времени.
- Это особенно важно для задач, где временные зависимости играют ключевую роль (например, финансовые данные или твои данные о сертификатах).

Как проверить:
```python
from statsmodels.graphics.tsaplots import plot_acf

# Остатки модели
residuals = test_data - forecast_values

# Построение ACF для остатков
plot_acf(residuals, lags=20)
plt.show()
```

Если ACF показывает значительные значения, попробуй улучшить модель или добавить компоненты ARIMA.

---

#### **b) Гетероскедастичность**
- Гетероскедастичность — это изменение дисперсии остатков со временем.
- Если она присутствует, это может указывать на проблемы с предсказанием (например, модель недооценивает изменения волатильности).

Как проверить:
- Построй график остатков.
- Если дисперсия увеличивается со временем, это гетероскедастичность.

Решения:
- Логарифмическое преобразование данных.
- Использование моделей с весами (например, Weighted Least Squares).

---

#### **c) Особенности для разных моделей**
- **Prophet**: Уже учитывает сезонность и тренды, но всё равно полезно проверить остатки.
- **LSTM**: Работает с нелинейными зависимостями, но требует большого объёма данных. Анализ остатков поможет понять, какие паттерны модель не смогла выявить.
- **XGBoost/CatBoost**: Эти модели чаще используются для регрессии или классификации, но могут применяться и для временных рядов. Анализ остатков аналогичен.

---

### **Итоговый план действий**
1. Для классических моделей (ARIMA/SARIMA):
   - Анализируй ACF/PACF для выбора параметров.
   - Проверяй стационарность и гетероскедастичность.
   - Используй критерии AIC/BIC для сравнения моделей.

2. Для моделей ML (Prophet, LSTM, XGBoost):
   - Анализируй остатки на наличие автокорреляции и гетероскедастичности.
   - Если проблемы есть, пробуй комбинировать модели (например, Prophet + ARIMA).

Если у тебя есть конкретные данные, можем вместе провести такой анализ! 😊

## 🔁 Основные эффективные методы кросс-валидации для временных рядов

### 1. **TimeSeriesSplit (раздвижное окно, expanding window)**
📦 **Что это:** встроен в `sklearn.model_selection.TimeSeriesSplit`.

🔧 **Как работает:**
- Данные делятся на несколько последовательных train/test-сплитов.
- Размер **train увеличивается**, а **test-окно фиксировано**.

🔍 **Пример:**

```python
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(data):
    X_train, X_test = data[train_index], data[test_index]
```

✅ **Плюсы:** простой, учитывает временную последовательность, можно контролировать размер теста.

---

### 2. **Expanding Window Cross-Validation**
📦 **Что это:** по сути — то же, что и `TimeSeriesSplit`.

🔧 **Как работает:**
- Train-окно **расширяется**, test — фиксировано.
- Пример:  
  - `train = [0:50], test = [51:60]`  
  - `train = [0:60], test = [61:70]`  
  - и т.д.

🧠 Подходит для моделей, где важно накопление информации (например, ARIMA, Prophet, CatBoost).

---

### 3. **Rolling Window Cross-Validation**
📦 **Что это:** окно **одинакового размера**, скользит по времени.

🔧 **Как работает:**
- И `train`, и `test` имеют **фиксированный размер**, но **оба окна двигаются**.
- Пример:
  - `train = [0:50], test = [51:60]`
  - `train = [10:60], test = [61:70]`

✅ **Плюсы:** хорошо выявляет, как модель работает в различных временных отрезках (особенно в условиях "концептуального сдвига").

❗ **Минус:** может теряться информация в старых данных, зато тренирует модель на более "свежем" контексте.

---

### 4. **Blocked K-Fold**
📦 **Что это:** KFold, но **блоками по времени**, без перемешивания.

🔧 **Как работает:**
- Делит данные на K последовательных блоков.
- Используется, когда данные **стационарны** и можно разрезать на равные части.

❗ Использовать **только если нет сильной зависимости во времени (что редко)**.

---

### 5. **Walk Forward Validation (WFV)**
📦 **Что это:** золотой стандарт для прогнозирования во времени.

🔧 **Как работает:**
- После каждого прогноза модель **дообучается на новом куске**.
- Можно использовать `expanding` или `rolling` подход к train.

📈 **Идеален для симуляции реального прогноза** в продакшене: модель тренируется до текущего момента и прогнозирует следующий шаг.

✅ **Плюсы:** максимально приближено к реальности.

---

### 6. **Purged K-Fold + Embargo** (если есть лейблы с лагом)
📦 Для ситуаций, где **целевая переменная доступна позже**, и будущее может случайно попасть в трейновую выборку.

🔧 Используется в финансовом time series и fraud detection:
- Purging — удаляем примыкающие записи между train и test.
- Embargo — делаем "буфер" между train и test.

📌 **Уместно при наличии "утечки данных по времени".**

---

## 💡 Что выбрать?
| Задача                                | Рекомендованная валидация               |
|--------------------------------------|-----------------------------------------|
| Классическое прогнозирование         | **Walk Forward Validation**, Expanding Window |
| Проверка устойчивости модели         | **Rolling Window Cross-Validation**     |
| Сравнение моделей                    | **TimeSeriesSplit**                     |
| Финансовые данные, лаг в таргете     | **Purged K-Fold + Embargo**             |

---

Хочешь, могу показать пример `Walk Forward Validation` в Python с `Prophet` или `CatBoost`?

### **1. Forward Chaining (Rolling Origin Cross-Validation)**

Это один из самых популярных и эффективных методов для временных рядов. Идея в том, чтобы постепенно увеличивать размер обучающей выборки, а тестовую выборку брать из следующих за ней данных.

#### Как это работает:
1. Разделите данные на последовательные блоки.
2. Для каждого шага:
   - Обучите модель на данных до определённого момента времени.
   - Проверьте её на данных после этого момента.
3. Повторяйте этот процесс, сдвигая " начало тестовой выборки" вперёд.

#### Пример:
Пусть у нас есть данные за 10 дней. Мы можем сделать следующие шаги:
- Шаг 1: Обучение на днях 1–3, тестирование на дне 4.
- Шаг 2: Обучение на днях 1–4, тестирование на дне 5.
- Шаг 3: Обучение на днях 1–5, тестирование на дне 6.
- И т.д.

#### Преимущества:
- Учитывает последовательность данных.
- Позволяет оценить стабильность модели при росте объёма данных.

#### Недостатки:
- Может быть вычислительно затратным для больших временных рядов.

---

### **2. Expanding Window**

МетодExpanding Window — это специальный случай Forward Chaining, где обучающая выборка постепенно увеличивается.

#### Как это работает:
1. Начните с маленькой обучающей выборки (например, первые 10% данных).
2. Добавляйте новые данные в обучающую выборку на каждом шаге.
3. Тестируйте модель на следующем временном интервале.

#### Пример:
- Шаг 1: Обучение на днях 1–3, тестирование на дне 4.
- Шаг 2: Обучение на днях 1–4, тестирование на дне 5.
- Шаг 3: Обучение на днях 1–5, тестирование на дне 6.

#### Преимущества:
- Хорошо подходит для моделей, которые улучшаются с увеличением объёма данных.
- Учитывает все предыдущие данные.

#### Недостатки:
- Может быть медленным, если модель долго обучается.

---

### **3. Sliding Window**

МетодSliding Window — это аналог Expanding Window, но обучающая выборка имеет фиксированный размер и "скользит" по данным.

#### Как это работает:
1. Выберите размер окна (например, 30 дней).
2. На каждом шаге:
   - Обучите модель на текущем окне данных.
   - Проверьте её на следующем временном интервале.
3. Сдвиньте окно на один шаг вперёд.

#### Пример:
- Шаг 1: Обучение на днях 1–3, тестирование на дне 4.
- Шаг 2: Обучение на днях 2–4, тестирование на дне 5.
- Шаг 3: Обучение на днях 3–5, тестирование на дне 6.

#### Преимущества:
- Более быстрый, чем Expanding Window, так как не требуется хранить всё предыдущее множество данных.
- Подходит для моделей с локальными паттернами.

#### Недостатки:
- Могут быть проблемы, если важные паттерны возникают только на длинных временных интервалах.

---

### **4. Blocked Time Series Split**

Этот метод разбивает данные на непересекающиеся блоки фиксированного размера. Каждый блок используется как отдельная пара обучающих и тестовых данных.

#### Как это работает:
1. Разделите данные на блоки равной длины.
2. Для каждого блока:
   - Используйте предыдущие блоки как обучающую выборку.
   - Используйте текущий блок как тестовую выборку.

#### Пример:
- Блок 1: Обучение на днях 1–10, тестирование на днях 11–20.
- Блок 2: Обучение на днях 1–20, тестирование на днях 21–30.
- Блок 3: Обучение на днях 1–30, тестирование на днях 31–40.

#### Преимущества:
- Хорошо подходит для данных с явной сезонностью.
- Учитывает зависимости между блоками.

#### Недостатки:
- Может быть трудно выбрать правильный размер блоков.

---

### **5. Purged and Embargoed Cross-Validation**

Этот метод используется для предотвращения утечки информации между обучающими и тестовыми данными. Особенно полезен для финансовых данных или других случаев, где автокорреляция высока.

#### Как это работает:
1. Разделите данные на обучающие и тестовые блоки.
2. Введите "запрещённую зону" (embargo) между обучающими и тестовыми данными, чтобы предотвратить перекрёстное влияние.
3. Обучайте модель только на данных вне запрещённой зоны.

#### Пример:
- Обучающие данные: дни 1–10.
- Запрещённая зона: дни 11–12.
- Тестовые данные: дни 13–20.

#### Преимущества:
- Предотвращает утечку информации.
- Подходит для данных с высокой автокорреляцией.

#### Недостатки:
- Может быть сложным для реализации.

---

### **Рекомендации**

1. **Основной выбор**: Forward Chaining (или его варианты, такие как Expanding Window и Sliding Window) — это наиболее универсальные методы для большинства задач.
2. **Для сезонных данных**: используй Blocked Time Series Split.
3. **Для данных с высокой автокорреляцией**: рассмотри Purged and Embargoed Cross-Validation.

Если у тебя есть конкретные данные, можем вместе выбрать оптимальный метод! 😊